# Fetching Student Mobile Numbers From Master Data

In [ ]:
import pandas as pd


# List of file paths to process
file_paths = [
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 3Jan.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 4Jan.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 5Jan.1.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 5Jan.2.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 6Jan.1.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 6Jan.2.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 6Jan.3.xlsx',
    r'D:\ppc\STUDE MASTER\Pariksha Pe Charcha 2024 Student 7Jan.xlsx',
    r'D:\ppc\STUDE MASTER\Student(1).xlsx',
    r'D:\ppc\STUDE MASTER\Student(1) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(1) - 3.xlsx',
    r'D:\ppc\STUDE MASTER\Student(1) - 4.xlsx',
    r'D:\ppc\STUDE MASTER\Student(2).xlsx',
    r'D:\ppc\STUDE MASTER\Student(2) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(2) - 3.xlsx',
    r'D:\ppc\STUDE MASTER\Student(2) - 4.xlsx',
    r'D:\ppc\STUDE MASTER\Student(2) - 5.xlsx',
    r'D:\ppc\STUDE MASTER\Student(3).xlsx',
    r'D:\ppc\STUDE MASTER\Student(3) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(4).xlsx',
    r'D:\ppc\STUDE MASTER\Student(4) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(5).xlsx',
    r'D:\ppc\STUDE MASTER\Student(5) -2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(6).xlsx',
    r'D:\ppc\STUDE MASTER\Student(6) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(7).xlsx',
    r'D:\ppc\STUDE MASTER\Student(7) - 2.xlsx',
    r'D:\ppc\STUDE MASTER\Student(8).xlsx',
    r'D:\ppc\STUDE MASTER\Student(9).xlsx',
    r'D:\ppc\STUDE MASTER\Student(9) - 2.xlsx',
    r'D:\ppc\Pariksha Pe Charcha 2024 Parent Type.xlsx',
    r'D:\ppc\Teacher Master (1).xlsx'
]


# Load data from the State Students' Excel file
file1_path = r'D:\ppc\01 April\3. CHATTISGARH_STUDENTS_HINDI_397241_504630 (107390).xlsx'
df1 = pd.read_excel(file1_path)

# Print the initial data in df1
print("Initial data in the Students State' Excel file:")
print(df1.head())

# Create a dictionary to map 'User ID' to a list of 'Student Mobile Numbers' from each file
mobile_numbers_dict = {}

# Iterate through each file path
for file_path in file_paths:
    print(f"\nProcessing file: {file_path}")
    
    # Load data from the current Excel file
    df = pd.read_excel(file_path)
    
    # Check if 'User ID', 'Student Mobile Number', 'Student eMail ID', 'Student First Name', and 'Student Last Name' columns exist in df
    required_columns = ['User ID', 'Student Mobile Number', 'Student eMail ID', 'Student First Name', 'Student Last Name']
    if all(col in df.columns for col in required_columns):
        print("Creating mobile numbers dictionary...")
        # Update mobile_numbers_dict with 'User ID' and list of 'Student Mobile Numbers' from df
        for index, row in df.iterrows():
            user_id = row['User ID']
            mobile_number = row['Student Mobile Number']
            email_id = row['Student eMail ID']
            first_name = row['Student First Name']
            last_name = row['Student Last Name']
            
            if user_id not in mobile_numbers_dict:
                mobile_numbers_dict[user_id] = {
                    'Mobile Numbers': [mobile_number],
                    'Email IDs': [email_id],
                    'First Names': [first_name],
                    'Last Names': [last_name]
                }
            else:
                mobile_numbers_dict[user_id]['Mobile Numbers'].append(mobile_number)
                mobile_numbers_dict[user_id]['Email IDs'].append(email_id)
                mobile_numbers_dict[user_id]['First Names'].append(first_name)
                mobile_numbers_dict[user_id]['Last Names'].append(last_name)
    else:
        print(f"Required columns not found in {file_path}. Skipping operation.")

# Function to prioritize mapping based on hierarchy
def map_mobile_numbers(row):
    user_id = row['User ID']
    
    # Check if user_id exists in mobile_numbers_dict
    if user_id in mobile_numbers_dict:
        mobile_numbers = mobile_numbers_dict[user_id]['Mobile Numbers']
        email_ids = mobile_numbers_dict[user_id]['Email IDs']
        first_names = mobile_numbers_dict[user_id]['First Names']
        last_names = mobile_numbers_dict[user_id]['Last Names']

        # Check if User ID has multiple mobile numbers
        if len(mobile_numbers) > 1:
            # Check if Student eMail ID can help in accuracy
            if len(set(email_ids)) == 1:
                return mobile_numbers[0]
            else:
                # Check if Student First Name and Last Name can help in accuracy
                student_name = row["Student's Name"]
                for i in range(len(first_names)):
                    # Check for NaN values before concatenating strings
                    if pd.notna(first_names[i]) and pd.notna(last_names[i]) and str(first_names[i]) + ' ' + str(last_names[i]) == student_name:
                        return mobile_numbers[i]
                # If no match found, return the first mobile number as a default
                return mobile_numbers[0]
        else:
            return mobile_numbers[0]
    else:
        return "N/A"  # Return a default value indicating user_id not found

# Update df1 with values from mobile_numbers_dict using the mapping function
df1['Student Mobile Number'] = df1.apply(map_mobile_numbers, axis=1)

# Save the updated DataFrame back to the Bihar students' Excel file
df1.to_excel(file1_path, index=False)
print(f"\nChanges saved to {file1_path}")

print("\nAll operations completed.")
